# Capstone Project - The Battle of the Neighborhoods (Week 1)

Joao Rocha´s Applied Data Science Capstone by IBM/Coursera

##### Table of contents
- Introduction: Business Problem

- Data

##### Introduction: Business Problem 

In this project, i am going to analyze the cities of Lisbon and Oporto, to determine between them, which one is more homogeneous in terms of most common venues, in order to help tourists decide which one they are going to visit in a trip to Portugal, and might feel more comfortable doing so

I will apply some Data Science techniques, such as Clustering by K-Means, and visualization tools like Folium, to help understand better the purpose of the project. A final discussion and conclusion will determine which city is more promising

##### Data 

For this project, I will be using:

- Data from Wikipedia, about neighborhoods and their geolocation, from the cities of Lisbon and Oporto, assembled and saved as an excel spreadsheet and then loaded into the model as a data frame


- Afterwards will extract the venues from those locations from Foursquare API


- Then develop the model with Python tools, applying cluster analysis and data visualization techniques, to obtain the final conclusions

